In [26]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [30]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [27]:
path = "/Users/sanketmishra/desktop/practice/tensorflow/imdb.csv"
imdb_data = pd.read_csv(path)
imdb_data["sentiment"].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [28]:
#split the dataset  
#train dataset
train_reviews=imdb_data.review[:400]
train_sentiments=imdb_data.sentiment[:400]
#test dataset
test_reviews=imdb_data.review[400:800]
test_sentiments=imdb_data.sentiment[400:800]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(400,) (400,)
(400,) (400,)


In [31]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

{'than', 'ain', 'own', 'above', "shan't", 'themselves', 'will', "needn't", 'been', 'hers', 'doesn', 'as', 'out', 're', 'aren', 'that', 'such', 'haven', "she's", 'into', "couldn't", 'then', 'by', 'no', 'ours', 'only', 'all', 've', "you'd", 'further', 'isn', 'our', "isn't", 'm', "mightn't", 'shan', 'if', "it's", "mustn't", 'theirs', 'be', 'mightn', "weren't", 'very', "haven't", 'not', 'more', 'having', "doesn't", 'because', 'they', 'of', 'has', 'an', 'couldn', 'is', "wouldn't", "that'll", 'off', 'but', 's', 'just', 'how', 'nor', 'during', 'with', 'where', "you're", 'have', 'so', "hadn't", 'its', 'again', 'shouldn', 'until', "aren't", 'against', "should've", "didn't", 'why', 'do', 'up', 'me', 'or', 'while', 'through', "you'll", 'too', 'ourselves', "won't", 'this', 'should', 'now', 'the', "you've", 'about', 'had', 'y', 'am', 'them', "hasn't", 'being', 'ma', 'she', 'wouldn', 'here', 'who', 'below', 'whom', 'after', 'him', 'mustn', 'o', 'won', 'and', 'himself', 'did', 'there', 'each', 'don',

In [36]:
norm_train_reviews=imdb_data.review[:400]
norm_train_reviews[0]
#Normalized test reviews
norm_test_reviews=imdb_data.review[400:800]
norm_test_reviews[600]
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

BOW_cv_train: (400, 98368)
BOW_cv_test: (400, 98368)
